# env

In [ ]:
//// test

open testing

## rust

### var_error

In [ ]:
nominal var_error =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::env::VarError\")>]\n#endif\ntype std_env_VarError = class end"
        $'' : $'std_env_VarError'
    )

### get_environment_variable_comptime

In [ ]:
inl get_environment_variable_comptime (var : string) : string =
    run_target_args (fun () => var) function
        | Rust _ => fun var =>
            open rust.rust_operators
            !\($'"option_env\!(\\\"" + !var + "\\\").unwrap_or(\\\"\\\")"')
            |> sm'.ref_to_std_string
            |> sm'.from_std_string
        | target => fun _ => null ()

## python

### os_environ

In [ ]:
nominal os_environ = any

In [ ]:
inl os_environ () : os_environ =
    backend_switch {
        Fsharp = fun () =>
            open python_operators
            global "type IOsEnviron = abstract environ: x: unit -> obj"
            inl os : $'IOsEnviron' = python.import_all "os"
            !\($'"!os.environ"') : os_environ
        Python = fun () =>
            global "import os"
            $'os.environ' : os_environ
    }

In [ ]:
inl environ_get (key : string) (os_environ : os_environ) : string =
    backend_switch {
        Fsharp = fun () =>
            open python_operators
            !\\(key, $'"!os_environ.get($0)"') : string
        Python = fun () =>
            $'!os_environ.get(!key)' : string
    }

## env

### get_environment_variable

In [ ]:
let get_environment_variable (var : string) : string =
    run_target_args (fun () => var) function
        | Rust (Native) => fun var =>
            inl var = join var
            open rust.rust_operators
            !\\(var, $'"std::env::var(&*$0)"')
            |> fun x => x : resultm.result' sm'.std_string var_error
            |> resultm.map' sm'.from_std_string
            |> resultm.unwrap_or' (join "")
        | Fsharp (Native) => fun var =>
            var
            |> $'System.Environment.GetEnvironmentVariable'
            |> optionm'.of_obj
            |> optionm'.unbox
            |> optionm'.default_value ""
        | TypeScript _ => fun var =>
            open typescript_operators
            !\\(var, $'"process.env[$0] ?? \\\"\\\""')
        | Python _ | Cuda _ => fun var =>
            os_environ ()
            |> environ_get var
            |> optionm'.of_obj
            |> optionm'.unbox
            |> optionm'.default_value ""
        | target => fun var => failwith $'$"env.get_environment_variable / target: {!target} / var: {!var}"'

In [ ]:
//// test
///! fsharp
///! cuda
///! rust
///! typescript
///! python

"PATH"
|> get_environment_variable
|> sm'.length
|> fun x =>
    if x > 0i32
    then 1
    else 0
    |> _assert_ne 0i32

.py output (Cuda):
__assert_ne / actual: 1 / expected: 0

.rs output:
__assert_ne / actual: 1 / expected: 0

.ts output:
__assert_ne / actual: 1 / expected: 0

.py output:
__assert_ne / actual: 1 / expected: 0



.fsx output:
__assert_ne / actual: 1 / expected: 0


### get_entry_assembly_name

In [ ]:
let get_entry_assembly_name () : string =
    run_target function
        | Rust _ => fun () => (join "CARGO_PKG_NAME") |> get_environment_variable
        | Fsharp _ => fun () => $'System.Reflection.Assembly.GetEntryAssembly().GetName().Name'
        | target => fun () => failwith $'$"env.get_entry_assembly_name / target: {!target}"'

### append_path

In [ ]:
inl append_path (path : string) : option string =
    inl env_path = "PATH" |> get_environment_variable
    if env_path = ""
    then None
    else
        inl env_sep =
            if platform.is_windows ()
            then ";"
            else ":"
        Some $'$"{!path}{!env_sep}{!env_path}"'